# Classifying Malaria Cell Images using Keras + Tensorflow

This example has been adapted from: https://towardsdatascience.com/deep-learning-to-identify-malaria-cells-using-cnn-on-kaggle-b9a987f55ea5

## Importing required libraries

We first begin by importing the python libraries that we require. For this tutorial, we would be using Keras over Tensorflow.


In [4]:
import cv2
import os
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.models import Sequential
from sklearn.model_selection import train_test_split

np.random.seed(1000)
%matplotlib inline

ModuleNotFoundError: No module named 'PIL'